## Project Name: **BrochureGenie**

### Description:
**BrochureGenie** is an intelligent brochure generation platform that automatically creates professional, visually appealing brochures for companies using just their name and website. It extracts key business insights, branding elements, and offerings from the company’s site and turns them into brochures tailored for potential clients, investors, and recruits.


In [94]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import google.generativeai as genai


In [95]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=gemini_api_key)

In [96]:
API_KEY = gemini_api_key
if not API_KEY:
    raise ValueError("GEMINI_API_KEY environment variable is not set.")

MODEL_NAME = "gemini-2.0-flash"

In [97]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [98]:
link = "https://www.geeksforgeeks.org/"

In [99]:
data = Website(link)
print(data.title)
print(data.text)

GeeksforGeeks | Your All-in-One Learning Portal
Courses
Tutorials
Practice
Data Structure
Java
Python
HTML
Interview Preparation
DSA
Practice Problems
C
C++
Java
Python
JavaScript
Data Science
Machine Learning
Courses
Linux
DevOps
SQL
Web Development
System Design
Aptitude
GfG Premium
Hello, What Do You Want To Learn?
Full Stack Live Classes
DSA: Basic To Advanced Course
Master DS & ML
Explore
Data Structure and Algorithms
View more
Practice DSA
View more
AI ML & Data Science
View more
Web Development
View more
Python
View more
Machine Learning
View more
System Design
View more
DevOps
View more
Interested in advertising with us?
Get in touch
Courses
View All
4.4
DSA to Development: A Complete Guide
Beginner to Advance
589k+
interested Geeks
Explore now
4.7
JAVA Backend Development - Live
Intermediate and Advance
312k+
interested Geeks
Explore now
4.9
Tech Interview 101 - From DSA to System Design for Working Professionals
Beginner to Advance
337k+
interested Geeks
Explore now
4.7
Full 

In [100]:
data.get_contents()

'Webpage Title:\nGeeksforGeeks | Your All-in-One Learning Portal\nWebpage Contents:\nCourses\nTutorials\nPractice\nData Structure\nJava\nPython\nHTML\nInterview Preparation\nDSA\nPractice Problems\nC\nC++\nJava\nPython\nJavaScript\nData Science\nMachine Learning\nCourses\nLinux\nDevOps\nSQL\nWeb Development\nSystem Design\nAptitude\nGfG Premium\nHello, What Do You Want To Learn?\nFull Stack Live Classes\nDSA: Basic To Advanced Course\nMaster DS & ML\nExplore\nData Structure and Algorithms\nView more\nPractice DSA\nView more\nAI ML & Data Science\nView more\nWeb Development\nView more\nPython\nView more\nMachine Learning\nView more\nSystem Design\nView more\nDevOps\nView more\nInterested in advertising with us?\nGet in touch\nCourses\nView All\n4.4\nDSA to Development: A Complete Guide\nBeginner to Advance\n589k+\ninterested Geeks\nExplore now\n4.7\nJAVA Backend Development - Live\nIntermediate and Advance\n312k+\ninterested Geeks\nExplore now\n4.9\nTech Interview 101 - From DSA to Syst

In [101]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON format as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [102]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON format as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [103]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [104]:
print(get_links_user_prompt(data))

Here is the list of links on the website of https://www.geeksforgeeks.org/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.geeksforgeeks.org/
https://www.geeksforgeeks.org/learn-data-structures-and-algorithms-dsa-tutorial/
https://www.geeksforgeeks.org/explore
https://www.geeksforgeeks.org/c-programming-language/
https://www.geeksforgeeks.org/c-plus-plus/
https://www.geeksforgeeks.org/java/
https://www.geeksforgeeks.org/python-programming-language/
https://www.geeksforgeeks.org/javascript/
https://www.geeksforgeeks.org/data-science-for-beginners/
https://www.geeksforgeeks.org/machine-learning/
https://www.geeksforgeeks.org/courses
https://www.geeksforgeeks.org/linux-tutorial/
https://www.geeksforgeeks.org/devops-tutorial/
https://www.geeksforgeeks.org/sql-tutorial/
https://www.geeksforgeeks.org/w

In [105]:
def get_links(url):
    website = Website(url)
    
    prompt = (
        link_system_prompt + "\n\n" +  # Instruction to Gemini
        get_links_user_prompt(website)  # User-level prompt constructed from the Website object
    )
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    
    response = model.generate_content(prompt)
    
    cleaned = response.text.strip()
    if cleaned.startswith("```json"):
        cleaned = cleaned.removeprefix("```json").strip()
    if cleaned.endswith("```"):
        cleaned = cleaned.removesuffix("```").strip()
    
    try:
        result = json.loads(cleaned)
        return result
    except json.JSONDecodeError:
        print("Failed to parse cleaned JSON:", cleaned)
        return None


In [106]:
links = get_links(link)
print(links)

{'links': [{'type': 'homepage', 'url': 'https://www.geeksforgeeks.org/'}, {'type': 'about page', 'url': 'https://www.geeksforgeeks.org/about/'}, {'type': 'contact us', 'url': 'https://www.geeksforgeeks.org/about/contact-us/'}, {'type': 'advertise with us', 'url': 'https://www.geeksforgeeks.org/advertise-with-us/'}, {'type': 'corporate solutions', 'url': 'https://www.geeksforgeeks.org/gfg-corporate-solution/'}, {'type': 'campus training program', 'url': 'https://www.geeksforgeeks.org/campus-training-program/'}, {'type': 'courses', 'url': 'https://www.geeksforgeeks.org/courses'}, {'type': 'press release', 'url': 'https://www.geeksforgeeks.org/press-release/'}]}


### make the brochure

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result, links

In [ ]:
print(get_all_details(link))

{'links': [{'type': 'homepage', 'url': 'https://www.geeksforgeeks.org/'}, {'type': 'about page', 'url': 'https://www.geeksforgeeks.org/about/'}, {'type': 'contact us', 'url': 'https://www.geeksforgeeks.org/about/contact-us/'}, {'type': 'advertise with us', 'url': 'https://www.geeksforgeeks.org/advertise-with-us/'}, {'type': 'corporate solution', 'url': 'https://www.geeksforgeeks.org/gfg-corporate-solution/'}, {'type': 'campus training program', 'url': 'https://www.geeksforgeeks.org/campus-training-program/'}, {'type': 'courses', 'url': 'https://www.geeksforgeeks.org/courses'}]}


In [109]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [120]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)[0]
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [121]:
get_brochure_user_prompt("geeksforgeeks", link)

'You are looking at a company called: geeksforgeeks\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nGeeksforGeeks | Your All-in-One Learning Portal\nWebpage Contents:\nCourses\nTutorials\nPractice\nData Structure\nJava\nPython\nHTML\nInterview Preparation\nDSA\nPractice Problems\nC\nC++\nJava\nPython\nJavaScript\nData Science\nMachine Learning\nCourses\nLinux\nDevOps\nSQL\nWeb Development\nSystem Design\nAptitude\nGfG Premium\nHello, What Do You Want To Learn?\nFull Stack Live Classes\nDSA: Basic To Advanced Course\nMaster DS & ML\nExplore\nData Structure and Algorithms\nView more\nPractice DSA\nView more\nAI ML & Data Science\nView more\nWeb Development\nView more\nPython\nView more\nMachine Learning\nView more\nSystem Design\nView more\nDevOps\nView more\nInterested in advertising with us?\nGet in touch\nCourses\nView All\n4.4\nDSA to Development: A Complete